In [1]:
from typing import Iterable, Iterator, TypeVar, List, Sequence
from rl.function_approx import Gradient
import rl.markov_process as mp
from rl.markov_decision_process import NonTerminal
import numpy as np
from rl.approximate_dynamic_programming import ValueFunctionApprox
from rl.approximate_dynamic_programming import extended_vf
from rl.markov_process import (MarkovRewardProcess, NonTerminal,
                               FiniteMarkovRewardProcess, TransitionStep)
from rl.function_approx import AdamGradient
from rl.function_approx import LinearFunctionApprox
from rl.approximate_dynamic_programming import ValueFunctionApprox
from rl.approximate_dynamic_programming import NTStateDistribution
from typing import Iterable, TypeVar, Callable, Iterator, Sequence

from rl.markov_process import (MarkovRewardProcess, NonTerminal,
                               FiniteMarkovRewardProcess, TransitionStep)
import itertools
from rl.chapter3.simple_inventory_mdp_cap import *
import rl.iterate as iterate
from rl.returns import returns
import rl.monte_carlo as mc
from rl.function_approx import learning_rate_schedule
import rl.td as td
import rl.td_lambda as td_lambda
from rl.approximate_dynamic_programming import ValueFunctionApprox
from rl.approximate_dynamic_programming import NTStateDistribution
import numpy as np
from rl.chapter2.simple_inventory_mrp import SimpleInventoryMRPFinite
from rl.chapter2.simple_inventory_mrp import InventoryState
from itertools import islice
from rl.distribution import Choose


from typing import TypeVar, Callable, Iterator, Sequence, Tuple, Mapping
from rl.function_approx import Tabular
from rl.distribution import Choose
from rl.markov_process import NonTerminal
from rl.markov_decision_process import (
    MarkovDecisionProcess, FiniteMarkovDecisionProcess,
    FiniteMarkovRewardProcess)
from rl.policy import FiniteDeterministicPolicy, FinitePolicy
from rl.approximate_dynamic_programming import QValueFunctionApprox
from rl.approximate_dynamic_programming import NTStateDistribution
import itertools
import rl.iterate as iterate
from rl.returns import returns
import rl.monte_carlo as mc
import rl.td as td
from rl.function_approx import learning_rate_schedule
from rl.dynamic_programming import V, value_iteration_result
from math import sqrt
from pprint import pprint
S = TypeVar('S')
A = TypeVar('A')

In [17]:
###### SARSA  ######
from operator import itemgetter
import itertools
from typing import Callable, Iterable, Iterator, TypeVar, Set, Sequence, Tuple

import numpy as np

from rl.approximate_dynamic_programming import ValueFunctionApprox, \
    QValueFunctionApprox, NTStateDistribution, extended_vf
from rl.distribution import Categorical
from rl.function_approx import LinearFunctionApprox, Weights
import rl.iterate as iterate
import rl.markov_process as mp
from rl.markov_decision_process import MarkovDecisionProcess
from rl.markov_decision_process import TransitionStep, NonTerminal
from rl.monte_carlo import greedy_policy_from_qvf
from rl.policy import Policy, DeterministicPolicy
from rl.experience_replay import ExperienceReplayMemory
def epsilon_greedy_action(
    q: QValueFunctionApprox[S, A],
    nt_state: NonTerminal[S],
    actions: Set[A],
    ϵ: float
) -> A:
    '''
    given a non-terminal state, a Q-Value Function (in the form of a
    FunctionApprox: (state, action) -> Value, and epislon, return
    an action sampled from the probability distribution implied by an
    epsilon-greedy policy that is derived from the Q-Value Function.
    '''
    greedy_action: A = max(
        ((a, q((nt_state, a))) for a in actions),
        key=itemgetter(1)
    )[0]
    return Categorical(
        {a: ϵ / len(actions) +
         (1 - ϵ if a == greedy_action else 0.) for a in actions}
    ).sample()
def glie_sarsa(
    mdp: FiniteMarkovDecisionProcess[S, A],
    states: NTStateDistribution[S],
    approx_0: QValueFunctionApprox[S, A],
    γ: float,
    ϵ_as_func_of_episodes: Callable[[int], float],
    max_episode_length: int
) -> Iterator[QValueFunctionApprox[S, A]]:
    q: QValueFunctionApprox[S, A] = approx_0
    yield q
    num_episodes: int = 0
    while True:
        num_episodes += 1
        ϵ: float = ϵ_as_func_of_episodes(num_episodes)
        state: NonTerminal[S] = states.sample()
        action: A = epsilon_greedy_action(
            q=q,
            nt_state=state,
            actions=set(mdp.actions(state)),
            ϵ=ϵ
        )
        steps: int = 0
        while isinstance(state, NonTerminal) and steps < max_episode_length:
            next_state, reward = mdp.step(state, action).sample()
            if isinstance(next_state, NonTerminal):
                next_action: A = epsilon_greedy_action(
                    q=q,
                    nt_state=next_state,
                    actions=set(mdp.actions(next_state)),
                    ϵ=ϵ
                )
                q = q.update([(
                    (state, action),
                    reward + γ * q((next_state, next_action))
                )])
                action = next_action
            else:
                q = q.update([((state, action), reward)])
            yield q
            steps += 1
            state = next_state

def glie_sarsa_learning_rate(
    mdp: MarkovDecisionProcess[S, A],
    start_state_distribution: NTStateDistribution[S],
    initial_func_approx: QValueFunctionApprox[S, A],
    gamma: float,
    epsilon_as_func_of_episodes: Callable[[int], float],
    max_episode_length: int
) -> Iterator[QValueFunctionApprox[S, A]]:
    return glie_sarsa(
        mdp=mdp,
        states=start_state_distribution,
        approx_0=initial_func_approx,
        γ=gamma,
        ϵ_as_func_of_episodes=epsilon_as_func_of_episodes,
        max_episode_length=max_episode_length
    )

capacity: int = 2
poisson_lambda: float = 1.0
holding_cost: float = 1.0
stockout_cost: float = 10.0

gamma: float = 0.9
gamma: float = 0.8  ### I CHOOSE THIS GAMMA FOR FAST CONVERGENCE 
## CLASSICAL SIMPLE INVENTORY
si_mrp = SimpleInventoryMRPFinite(
    capacity=capacity,
    poisson_lambda=poisson_lambda,
    holding_cost=holding_cost,
    stockout_cost=stockout_cost
)

si_mdp=SimpleInventoryMDPCap(
    capacity=capacity,
    poisson_lambda=poisson_lambda,
    holding_cost=holding_cost,
    stockout_cost=stockout_cost
)
all_states = si_mdp.non_terminal_states
true_vf = si_mrp.get_value_function_vec(gamma=gamma)


mc_episode_length_tol: float = 1e-6
num_episodes = 10000

td_episode_length: int = 100
initial_learning_rate: float = 0.03
half_life: float = 1000.0
exponent: float = 0.5

#### FUNCTION APPROXIMATION WITH STATE INDICATOR FEATURES  ####
ag = AdamGradient(
    learning_rate=0.05,
    decay1=0.9,
    decay2=0.999
)


### This is for tabular case, but we can generalize by changing our functions. 
### For example we can take x[0].state.on_hand as a parameter, or even lets say
### we can take x[0].state.on_hand^2 as a feature.
ffs=[]
for s in si_mdp.mapping.keys():
    for a in si_mdp.mapping[s].keys():
        ffs.append(lambda x,s=s,a=a:float(x[0].state==s.state and x[1]==a))

lfa = LinearFunctionApprox.create(
        feature_functions=ffs,
        adam_gradient=ag,
        direct_solve=False
)
episode_length=1000
sarsa: Iterable[QValueFunctionApprox[InventoryState,float]] = \
    glie_sarsa_learning_rate(
        mdp=si_mdp,
        start_state_distribution=Choose(all_states),
        gamma=gamma,
        initial_func_approx=lfa,
        max_episode_length=1000,
        epsilon_as_func_of_episodes=lambda x: 0.1)

import math
sarsa_rmse_vals=[]
sarsa_episodes: int = 100000
for i, sarsa_vf in enumerate(islice(sarsa, sarsa_episodes)):
    if i%2000==0:
        print(f"iteration : {i}")
print(f"sarsa QVF {sarsa_vf.weights.weights}")
vf_list = []
for s in si_mdp.mapping.keys():
    maxi = -math.inf
    for a in si_mdp.mapping[s].keys():
        if maxi<= sarsa_vf.evaluate([(s,a)])[0]:
            maxi=sarsa_vf.evaluate([(s,a)])[0]
    vf_list.append(maxi)
print(f"Sarsa Value Function : {vf_list}")
print(f"True VF : {true_vf}")

iteration : 0
iteration : 500
iteration : 1000
iteration : 1500
iteration : 2000
iteration : 2500
iteration : 3000
iteration : 3500
iteration : 4000
iteration : 4500
iteration : 5000
iteration : 5500
iteration : 6000
iteration : 6500
iteration : 7000
iteration : 7500
iteration : 8000
iteration : 8500
iteration : 9000
iteration : 9500
iteration : 10000
iteration : 10500
iteration : 11000
iteration : 11500
iteration : 12000
iteration : 12500
iteration : 13000
iteration : 13500
iteration : 14000
iteration : 14500
iteration : 15000
iteration : 15500
iteration : 16000
iteration : 16500
iteration : 17000
iteration : 17500
iteration : 18000
iteration : 18500
iteration : 19000
iteration : 19500
iteration : 20000
iteration : 20500
iteration : 21000
iteration : 21500
iteration : 22000
iteration : 22500
iteration : 23000
iteration : 23500
iteration : 24000
iteration : 24500
iteration : 25000
iteration : 25500
iteration : 26000
iteration : 26500
iteration : 27000
iteration : 27500
iteration : 2800

In [19]:
###### Q-LEARNING #####
def q_learning_learning_rate(
    mdp: MarkovDecisionProcess[S, A],
    start_state_distribution: NTStateDistribution[S],
    initial_func_approx: QValueFunctionApprox[S, A],
    gamma: float,
    epsilon: float,
    max_episode_length: int
) -> Iterator[QValueFunctionApprox[S, A]]:
    return td.q_learning(
        mdp=mdp,
        policy_from_q=lambda f, m: mc.epsilon_greedy_policy(
            q=f,
            mdp=m,
            ϵ=epsilon
        ),
        states=start_state_distribution,
        approx_0=initial_func_approx,
        γ=gamma,
        max_episode_length=max_episode_length
    )
capacity: int = 2
poisson_lambda: float = 1.0
holding_cost: float = 1.0
stockout_cost: float = 10.0

gamma: float = 0.9
gamma: float = 0.8  ### I CHOOSE THIS GAMMA FOR FAST CONVERGENCE 
## CLASSICAL SIMPLE INVENTORY
si_mrp = SimpleInventoryMRPFinite(
    capacity=capacity,
    poisson_lambda=poisson_lambda,
    holding_cost=holding_cost,
    stockout_cost=stockout_cost
)

si_mdp=SimpleInventoryMDPCap(
    capacity=capacity,
    poisson_lambda=poisson_lambda,
    holding_cost=holding_cost,
    stockout_cost=stockout_cost
)
all_states = si_mdp.non_terminal_states
true_vf = si_mrp.get_value_function_vec(gamma=gamma)


mc_episode_length_tol: float = 1e-6
num_episodes = 10000

td_episode_length: int = 100
initial_learning_rate: float = 0.03
half_life: float = 1000.0
exponent: float = 0.5

#### FUNCTION APPROXIMATION WITH STATE INDICATOR FEATURES  ####
ag = AdamGradient(
    learning_rate=0.05,
    decay1=0.9,
    decay2=0.999
)

### This is for tabular case, but we can generalize by changing our functions. 
### For example we can take x[0].state.on_hand as a parameter, or even lets say
### we can take x[0].state.on_hand^2 as a feature.
ffs=[]
for s in si_mdp.mapping.keys():
    for a in si_mdp.mapping[s].keys():
        ffs.append(lambda x,s=s,a=a:float(x[0].state==s.state and x[1]==a))
        
lfa = LinearFunctionApprox.create(
        feature_functions=ffs,
        adam_gradient=ag,
        direct_solve=False
)
episode_length=1000
q_lear: Iterable[QValueFunctionApprox[InventoryState,float]] = \
    q_learning_learning_rate(
        mdp=si_mdp,
        start_state_distribution=Choose(all_states),
        gamma=gamma,
        initial_func_approx=lfa,
        max_episode_length=1000,
        epsilon=0.1,
    )

sarsa_rmse_vals=[]
q_lear_episodes: int = 50000
for i, qlear_vf in enumerate(islice(q_lear, q_lear_episodes)):
    if i%2000==0:
        print(f"iteration : {i}")
print(f"qlearning QVF {qlear_vf.weights.weights}")

vf_list = []
for s in si_mdp.mapping.keys():
    maxi = -math.inf
    for a in si_mdp.mapping[s].keys():
        if maxi<= qlear_vf.evaluate([(s,a)])[0]:
            maxi=qlear_vf.evaluate([(s,a)])[0]
    vf_list.append(maxi)
print(f"Q Learning Value Function : {vf_list}")

print(f"True VF : {true_vf}")


iteration : 0
iteration : 2000
iteration : 4000
iteration : 6000
iteration : 8000
iteration : 10000
iteration : 12000
iteration : 14000
iteration : 16000
iteration : 18000
iteration : 20000
iteration : 22000
iteration : 24000
iteration : 26000
iteration : 28000
iteration : 30000
iteration : 32000
iteration : 34000
iteration : 36000
iteration : 38000
iteration : 40000
iteration : 42000
iteration : 44000
iteration : 46000
iteration : 48000
qlearning QVF [-26.58903029 -21.10830166 -20.45944982 -18.16895884 -13.32752454
 -12.8264834  -16.83930414 -14.60149306 -13.6732792  -14.6565989 ]
Q Learning Value Function : [-20.459449822593385, -13.327524537045528, -12.826483400169202, -14.60149306134891, -13.673279202942279, -14.656598902360578]
True VF : [-20.69406782 -13.25880178 -13.36758477 -14.25880178 -14.36758477
 -15.36758477]


In [12]:
# ############## IGNORE THIS PART  ##############
# from typing import Tuple, Callable, Sequence
# from rl.chapter11.control_utils import glie_mc_finite_learning_rate_correctness
# from rl.chapter11.control_utils import \
#     q_learning_finite_learning_rate_correctness
# from rl.chapter11.control_utils import \
#     glie_sarsa_finite_learning_rate_correctness
# from rl.chapter11.control_utils import compare_mc_sarsa_ql
# from rl.chapter3.simple_inventory_mdp_cap import SimpleInventoryMDPCap


# capacity: int = 2
# poisson_lambda: float = 1.0
# holding_cost: float = 1.0
# stockout_cost: float = 10.0

# si_mdp: SimpleInventoryMDPCap = SimpleInventoryMDPCap(
#     capacity=capacity,
#     poisson_lambda=poisson_lambda,
#     holding_cost=holding_cost,
#     stockout_cost=stockout_cost
# )

# gamma: float = 0.9
# mc_episode_length_tol: float = 1e-5
# num_episodes = 10000

# epsilon_as_func_of_episodes: Callable[[int], float] = lambda k: k ** -0.5
# q_learning_epsilon: float = 0.2

# td_episode_length: int = 100
# initial_learning_rate: float = 0.1
# half_life: float = 10000.0
# exponent: float = 1.0

# glie_mc_finite_learning_rate_correctness(
#     fmdp=si_mdp,
#     initial_learning_rate=initial_learning_rate,
#     half_life=half_life,
#     exponent=exponent,
#     gamma=gamma,
#     epsilon_as_func_of_episodes=epsilon_as_func_of_episodes,
#     episode_length_tolerance=mc_episode_length_tol,
#     num_episodes=num_episodes
# )

# glie_sarsa_finite_learning_rate_correctness(
#     fmdp=si_mdp,
#     initial_learning_rate=initial_learning_rate,
#     half_life=half_life,
#     exponent=exponent,
#     gamma=gamma,
#     epsilon_as_func_of_episodes=epsilon_as_func_of_episodes,
#     max_episode_length=td_episode_length,
#     num_updates=num_episodes * td_episode_length
# )

# q_learning_finite_learning_rate_correctness(
#     fmdp=si_mdp,
#     initial_learning_rate=initial_learning_rate,
#     half_life=half_life,
#     exponent=exponent,
#     gamma=gamma,
#     epsilon=q_learning_epsilon,
#     max_episode_length=td_episode_length,
#     num_updates=num_episodes * td_episode_length
# )

# num_episodes = 500
# plot_batch: int = 10
# plot_start: int = 0
# learning_rates: Sequence[Tuple[float, float, float]] = \
#     [(0.05, 1000000, 0.5)]

# compare_mc_sarsa_ql(
#     fmdp=si_mdp,
#     method_mask=[True, True, False],
#     learning_rates=learning_rates,
#     gamma=gamma,
#     epsilon_as_func_of_episodes=epsilon_as_func_of_episodes,
#     q_learning_epsilon=q_learning_epsilon,
#     mc_episode_length_tol=mc_episode_length_tol,
#     num_episodes=num_episodes,
#     plot_batch=plot_batch,
#     plot_start=plot_start
# )

GLIE MC Optimal Value Function with 10000 episodes
{NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -35.085719335513666,
 NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -27.71389758645516,
 NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -28.558080366141244,
 NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -29.06629022397233,
 NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -29.46907723115718,
 NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -30.36796015491581}
GLIE MC Optimal Policy with 10000 episodes
For State InventoryState(on_hand=0, on_order=0): Do Action 1
For State InventoryState(on_hand=0, on_order=1): Do Action 1
For State InventoryState(on_hand=0, on_order=2): Do Action 0
For State InventoryState(on_hand=1, on_order=0): Do Action 1
For State InventoryState(on_hand=1, on_order=1): Do Action 0
For State InventoryState(on_hand=2, on_order=0): Do Action 0

True Optimal Value Function
{NonTerminal(state=InventoryState(o

<Figure size 1100x700 with 1 Axes>